In [ ]:
import sys
import os
import logging
import pandas as pd
import datasets
from pprint import pprint
KEY = '2-OhioT1DM'
WORKSPACE_PATH = os.getcwd().split(KEY)[0]
print(WORKSPACE_PATH); os.chdir(WORKSPACE_PATH)
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')


SPACE = {
    'DATA_RAW': f'_Data/0-Data_Raw',
    'DATA_RFT': f'_Data/1-Data_RFT',
    'DATA_CASE': f'_Data/2-Data_CASE',
    'DATA_CFDATA': f'_Data/3-Data_CFDATA',
    'DATA_EXTERNAL': f'code/external',
    'DATA_HFDATA': f'_Data/5-Data_HFData',
    'CODE_FN': f'code/pipeline',
    'MODEL_ROOT': f'./_Model',
}
assert os.path.exists(SPACE['CODE_FN']), f'{SPACE["CODE_FN"]} not found'
print(SPACE['CODE_FN'])
sys.path.append(SPACE['CODE_FN'])

/home/jluo41/CGMLSM-Project/
code/pipeline


In [2]:
from recfldtkn.aidata_base.entry import EntryAIData_Builder
from recfldtkn.case_base.casefnutils.casefn import Case_Fn

 
CF_DataName = 'OhioT1DM-GluPred-CGM5MinEntry-10bfff5cd1e01faf'
CohortName_list = ['OhioT1DM']

######################## get the CF_DataName list
CF_DataName_list = [f'{CF_DataName}/{i}' for i in CohortName_list]
########################
CF_DataName_list

entry = EntryAIData_Builder(SPACE=SPACE)
dataset = entry.merge_one_cf_dataset(CF_DataName_list)
dataset

[WARNING:2025-07-08 16:04:01,971:(entry.py@137 recfldtkn.aidata_base.entry)]: No EntryInput Method for code/pipeline/fn/fn_aidata/entryinput/None.py
[INFO:2025-07-08 16:04:01,983:(entry.py@305 recfldtkn.aidata_base.entry)]: set up TriggerName: CGM5MinEntry
[INFO:2025-07-08 16:04:01,983:(entry.py@306 recfldtkn.aidata_base.entry)]: set up CF_Config: ['CohortName_list', 'CohortName_to_OneCohortArgs', 'CaseSettingInfo', 'Record_Proc_Config', 'Case_Proc_Config', 'TriggerCaseBaseName', 'TriggerCaseBaseName_to_CohortNameList', 'TriggerCaseBaseName_to_TriggerCaseBaseArgs', 'SPACE', 'CF_to_CFvocab']
[INFO:2025-07-08 16:04:01,984:(entry.py@112 recfldtkn.aidata_base.entry)]: Load Trigger Method for code/pipeline/fn/fn_case/trigger/CGM5MinEntry.py


Dataset({
    features: ['PID', 'ObsDT', 'CGMValueBf24h--tid', 'CGMValueAf2h--tid', 'CGMInfoBf24h-ModePercent', 'CGMInfoBf24h-ZeroPercent', 'CGMInfoBf24h-Geq400Percent', 'CGMInfoBf24h-Leq20Percent', 'CGMInfoBf24h-RecNum', 'CGMInfoAf2h-ModePercent', 'CGMInfoAf2h-ZeroPercent', 'CGMInfoAf2h-Geq400Percent', 'CGMInfoAf2h-Leq20Percent', 'CGMInfoAf2h-RecNum', 'CGMInfoAf2to8h-ModePercent', 'CGMInfoAf2to8h-ZeroPercent', 'CGMInfoAf2to8h-Geq400Percent', 'CGMInfoAf2to8h-Leq20Percent', 'CGMInfoAf2to8h-RecNum', 'YearOfBirth', 'Gender', 'MRSegmentID', 'DiseaseType_x', 'Date', 'DiseaseType_y', 'split', 'middle_first_date', 'late_first_date', 'time_bin', 'final_split', 'Selected'],
    num_rows: 58414
})

In [ ]:
df = dataset.to_pandas()
df

In [4]:
df['key'] = df['CGMValueBf24h--tid'].apply(lambda x: '-'.join([str(i) for i in x]))
vc = df['key'].value_counts()
print(vc.value_counts())

vc = vc[vc > 1]

dfx = df[df['key'].isin(vc.index.tolist())]

dfx['PID'].value_counts().index.tolist()
# TODO: describe the dataset so people can have a quick look

count
1    58414
Name: count, dtype: int64


[]

In [5]:
data_config = dataset.info.config_name 
CF_to_CFvocab = data_config['CF_to_CFvocab']


CFName = 'HM5MinStep'
interval_delta = pd.Timedelta(minutes=5)
idx2tkn = [pd.Timestamp('2022-01-01 00:00:00') + interval_delta * i for i in range(24 * 12)]
idx2tkn = [f'{i.hour:02d}:{i.minute:02d}' for i in idx2tkn]
tkn2idx = {tkn: idx for idx, tkn in enumerate(idx2tkn)}
CF_to_CFvocab = data_config['CF_to_CFvocab']
CF_to_CFvocab[CFName] = {'idx2tkn': idx2tkn, 'tkn2idx': tkn2idx}

CFName = 'CGMValue'
idx2tkn = ["PAD", "UNKNOWN", "MASK", '<start>', '<end>'] + [f'Other_{i}' for i in range(0, 5)] + [str(i) for i in range(10, 401)]
tkn2idx = {tkn: idx for idx, tkn in enumerate(idx2tkn)}
CF_to_CFvocab[CFName] = {'idx2tkn': idx2tkn, 'tkn2idx': tkn2idx}

In [6]:
# CF_to_CFvocab['HM5MinStep']['tkn2idx']
# CF_to_CFvocab['CGMValue']['tkn2idx']

In [7]:
pd.Series(dataset['final_split']).value_counts()

train      44275
test-id     9214
valid       4925
Name: count, dtype: int64

In [8]:
OneEntryArgs = {
    # ----------------- Input Part -----------------
    'Split_Part': {
        'SplitMethod': 'SplitFromColumns',
        'Split_to_Selection': {
            'train': {
                'Rules': [
                    ['final_split', '==', 'train'],
                ],
                'Op': 'and'
            },
            'valid': {
                'Rules': [
                    ['final_split', '==', 'valid'],
                ],
                'Op': 'and'
            },
            'test-id': {
                'Rules': [
                    ['final_split', '==', 'test-id'],
                ],
                'Op': 'and'
            },
        }
    },

    'Input_Part': {
        'EntryInputMethod': '1TknInStepWt5MinHM',
        'CF_list': [
            'CGMValueBf24h',
            'CGMValueAf2h',
           #  'CGMValueAf2to8h',
        ],
        'BeforePeriods': ['Bf24h'],
        'AfterPeriods': ['Af2h'],
        'TimeIndex': True, 
        'InferenceMode': False, # True, # True, # False, # True, 
        'TargetField': 'CGMValue',
        'TargetRange': [40, 400],
        # 'HM': None, 
        'HM': {'start': -24, 'unit': 'h', 'interval': '5m'},
    }, 


    'Output_Part': {
        'EntryOutputMethod': 'FutureSeq',
        'FutureStart': 289,
        'FuturePeriod': 12 * 8, 
        'selected_columns': [
            'PID', # 'ObsDT', 
            'input_ids', 
            'hm_ids', 
            'labels',
            'final_split',
        ],
        'set_transform': False,
        'num_proc': 4, 
    },
}

entry = EntryAIData_Builder(OneEntryArgs=OneEntryArgs, SPACE=SPACE)
entry.CF_to_CFvocab = CF_to_CFvocab


[INFO:2025-07-08 16:04:04,073:(entry.py@131 recfldtkn.aidata_base.entry)]: Load EntryInput Method for code/pipeline/fn/fn_aidata/entryinput/1TknInStepWt5MinHM.py
[INFO:2025-07-08 16:04:04,074:(entry.py@146 recfldtkn.aidata_base.entry)]: Load EntryOutput Method for code/pipeline/fn/fn_aidata/entryoutput/FutureSeq.py
[INFO:2025-07-08 16:04:04,075:(entry.py@163 recfldtkn.aidata_base.entry)]: Load Split Method for code/pipeline/fn/fn_aidata/split/SplitFromColumns.py


In [9]:
ds = dataset#.select(range(10))
Data = {'ds_case': ds}
Data = entry.setup_EntryFn_to_Data(Data, CF_to_CFvocab)

[INFO:2025-07-08 16:04:04,502:(entry.py@195 recfldtkn.aidata_base.entry)]: entry_fn_AIInputData: <function entry_fn_AIInputData at 0x79a9fbf81da0>
[INFO:2025-07-08 16:04:04,503:(entry.py@196 recfldtkn.aidata_base.entry)]: tfm_fn_AIInputData: <function tfm_fn_AIInputData at 0x79a9fbf82020>
[INFO:2025-07-08 16:04:04,504:(entry.py@197 recfldtkn.aidata_base.entry)]: entry_fn_AITaskData: <function entry_fn_AITaskData at 0x79a9fbf82200>
[INFO:2025-07-08 16:04:04,504:(entry.py@215 recfldtkn.aidata_base.entry)]: entry_fn_AITaskData is executed


Map (num_proc=4):   0%|          | 0/58414 [00:00<?, ? examples/s]

In [10]:
dataset

Dataset({
    features: ['PID', 'ObsDT', 'CGMValueBf24h--tid', 'CGMValueAf2h--tid', 'CGMInfoBf24h-ModePercent', 'CGMInfoBf24h-ZeroPercent', 'CGMInfoBf24h-Geq400Percent', 'CGMInfoBf24h-Leq20Percent', 'CGMInfoBf24h-RecNum', 'CGMInfoAf2h-ModePercent', 'CGMInfoAf2h-ZeroPercent', 'CGMInfoAf2h-Geq400Percent', 'CGMInfoAf2h-Leq20Percent', 'CGMInfoAf2h-RecNum', 'CGMInfoAf2to8h-ModePercent', 'CGMInfoAf2to8h-ZeroPercent', 'CGMInfoAf2to8h-Geq400Percent', 'CGMInfoAf2to8h-Leq20Percent', 'CGMInfoAf2to8h-RecNum', 'YearOfBirth', 'Gender', 'MRSegmentID', 'DiseaseType_x', 'Date', 'DiseaseType_y', 'split', 'middle_first_date', 'late_first_date', 'time_bin', 'final_split', 'Selected'],
    num_rows: 58414
})

In [11]:
from datasets import DatasetInfo

DataName = 'OhioT1DM-Split_v0628'
path = os.path.join(SPACE['DATA_HFDATA'], f'{DataName}')
    

ds_tfm = Data['ds_tfm']
dataset_info = DatasetInfo.from_dict({'config_name': data_config})
ds_tfm.info.update(dataset_info)

ds_tfm

Dataset({
    features: ['PID', 'final_split', 'input_ids', 'hm_ids', 'labels'],
    num_rows: 58414
})

In [12]:

dataset = ds_tfm
split_to_dataset = entry.split_cf_dataset(dataset)
split_to_dataset.save_to_disk(path)


[INFO:2025-07-08 16:04:15,838:(base.py@520 recfldtkn.base)]: condition: ['final_split', '==', 'train']: to select: 44275 out of 58414
[INFO:2025-07-08 16:04:15,856:(base.py@520 recfldtkn.base)]: condition: ['final_split', '==', 'valid']: to select: 4925 out of 58414
[INFO:2025-07-08 16:04:15,862:(base.py@520 recfldtkn.base)]: condition: ['final_split', '==', 'test-id']: to select: 9214 out of 58414


Saving the dataset (0/1 shards):   0%|          | 0/44275 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4925 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9214 [00:00<?, ? examples/s]

In [13]:
split_to_dataset

DatasetDict({
    train: Dataset({
        features: ['PID', 'final_split', 'input_ids', 'hm_ids', 'labels'],
        num_rows: 44275
    })
    valid: Dataset({
        features: ['PID', 'final_split', 'input_ids', 'hm_ids', 'labels'],
        num_rows: 4925
    })
    test-id: Dataset({
        features: ['PID', 'final_split', 'input_ids', 'hm_ids', 'labels'],
        num_rows: 9214
    })
})